Se importan los paquetes necesarios

In [298]:
import numpy as np
import pandas as pd

Se importan los datos como pandas DataFrame

In [299]:
sharks = pd.read_csv("data/attacks.csv",encoding = "ISO-8859-1")

Se exploran los datos

In [300]:
sharks.shape

(25723, 24)

In [301]:
sharks.sample(5)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
2083,1999.12.14,14-Dec-1999,1999.0,Invalid,USA,California,"Off Ventura, Anacapa & Santa Cruz Islands",Scuba diving,Joo Whan Hong,M,...,Shark involvement not confirmed,R. Collier,1999.12.14-Hong.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1999.12.14,1999.12.14,4220.0,NaN,NaN
1082,2009.10.18,18-Oct-2009,2009.0,Unprovoked,PANAMA,Bocas,Playa La Cabaña,Wading,female,F,...,NaN,A. Blaker,2009.10.18-Panama.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2009.10.18,2009.10.18,5221.0,NaN,NaN
9654,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5520,1903.06.21,21-Jun-1903,1903.0,Unprovoked,AUSTRALIA,New South Wales,Georges River,Swimming,William Price,M,...,NaN,"The Advertiser, 6/22/1903",1903.06.21-Price.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1903.06.21,1903.06.21,783.0,NaN,NaN
18694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [302]:
sharks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [303]:
sharks.isna().sum()

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

Se comprueba que es un DataFrame muy sucio con una gran cantidad de nulos

Se cambia el nombre de todas las columnas del DataFrame para simplificar

In [305]:
nuevas_columna = {columna:(columna.replace(" ", "_").replace(".", "_").lower() if columna[-1]!=" "
                  else columna.replace(" ", "_").replace(".", "_").lower()[:-1]) for columna in list(sharks.keys())}

In [306]:
sharks.rename(columns=nuevas_columna, inplace=True)

In [307]:
sharks.sample(5)

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
5049,1930.02.15,15-Feb-1930,1930.0,Unprovoked,AUSTRALIA,Victoria,"Middle Brighton, Port Phillip",Diving off pier & treading water,Norman Clark,M,...,White shark 4.9 m [16'],V.M. Coppleson.V2.(1933); V.M. Coppleson (1958...,1930.02.15-Clarke.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1930.02.15,1930.02.15,1254.0,NaN,NaN
6409,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4556,1948.00.00.c,1948,1948.0,Unprovoked,NEW GUINEA,Morobe Province,Finschafen,"Fishing, holding fish in right hand",male,M,...,NaN,"A.D. Campbell; A. Bleakley; A. M. Rapson, p.148",1948.00.00.c-Finschafen.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1948.00.00.c,1948.00.00.c,1747.0,NaN,NaN


Ahora se va a proceder a la limpieza de los datos.

Lo primero es eliminar todas las filas con datos repetidos

In [308]:
sharks.drop_duplicates(inplace=True)

A continuación se van a eliminar todas aquellas filas donde todas las celdas sean nulos y todas las columnas donde todos los valores sean nulos

In [309]:
sharks.dropna(axis=0, how='all', inplace=True)
sharks.dropna(axis=1, how='all', inplace=True)

In [310]:
sharks.shape

(6311, 24)

Con este proceso se han limpiado bastantes filas.

In [311]:
sharks[(sharks["case_number"]=="0")].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8 entries, 6302 to 6309
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   case_number             8 non-null      object 
 1   date                    0 non-null      object 
 2   year                    0 non-null      float64
 3   type                    0 non-null      object 
 4   country                 0 non-null      object 
 5   area                    0 non-null      object 
 6   location                0 non-null      object 
 7   activity                0 non-null      object 
 8   name                    0 non-null      object 
 9   sex                     0 non-null      object 
 10  age                     0 non-null      object 
 11  injury                  0 non-null      object 
 12  fatal_(y/n)             0 non-null      object 
 13  time                    0 non-null      object 
 14  species                 0 non-null      

In [312]:
sharks[(sharks["case_number"]=="0")].shape

(8, 24)

Se comprueba que en aquellas columnas donde "case_number" es nulo, todo el resto de columnas salvo "original__order" son nulas también, por lo que no aportan información útil y se van a eliminar.

In [313]:
sharks.drop(sharks[(sharks["case_number"]=="0")].index,inplace= True)

In [314]:
sharks.isna().sum()

case_number                  1
date                         1
year                         3
type                         5
country                     51
area                       456
location                   541
activity                   545
name                       211
sex                        566
age                       2832
injury                      29
fatal_(y/n)                540
time                      3355
species                   2839
investigator_or_source      18
pdf                          1
href_formula                 2
href                         1
case_number_1                1
case_number_2                1
original_order               1
unnamed:_22               6302
unnamed:_23               6301
dtype: int64

Se comprueba que el DataFrame está mucho más limpio. Viendo la información del DF se ve que las columnas "unnamed:_22" y "unnamed:_23" casi todos los valores son NaN. Se va a verificar cuales son aquellos valores no nulos y posteriormente eliminar las columnas completas

In [315]:
sharks.shape

(6303, 24)

In [316]:
sharks[sharks["unnamed:_22"].notna()].sample()

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
1478,2006.05.27,27-May-2006,2006.0,Unprovoked,USA,Hawaii,"North Shore, O'ahu",Surfing,Bret Desmond,M,...,NaN,R. Collier,2006.05.27-Desmond.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2006.05.27,2006.05.27,4825.0,stopped here,NaN


In [317]:
sharks[sharks["unnamed:_23"].notna()].sample(2)

,case_number,date,year,type,country,area,location,activity,name,sex,...,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order,unnamed:_22,unnamed:_23
5840,1878.09.14.R,Reported 14-Sep-1878,1878.0,Provoked,USA,Connecticut,"Branford, New Haven County",Fishing,Captain Pattison,M,...,NaN,"St. Joseph Herald, 9/14/1878",1878.09.14.R-Pattison.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1878.09.14.R,1878.09.14.R,463.0,NaN,change filename
4415,1952.03.30,30-Mar-1952,1952.0,Unprovoked,NETHERLANDS ANTILLES,Curacao,NaN,Went to aid of child being menaced by the shark,A.J. Eggink,M,...,"Bull shark, 2.7 m [9'] was captured & dragged ...","J. Randall, p.352 in Sharks & Survival; H.D. B...",1952.03.30-Eggink.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1952.03.30,1952.03.30,1888.0,NaN,Teramo


No hay ningún dato valioso en esas columnas por lo que se van a eliminar

In [318]:
sharks.drop(columns = ["unnamed:_22", "unnamed:_23"],axis=1,inplace= True)

In [319]:
sharks.sample(5)

,case_number,date,year,type,country,area,location,activity,name,sex,...,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number_1,case_number_2,original_order
1748,2003.09.00,Sep-2003,2003.0,Unprovoked,USA,Florida,"Boca Grande, Lee County",Wade-fishing,male,M,...,N,NaN,NaN,"B. Stout, News-Press, 9/24/2003",2003.09.00-wade-fisherman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2003.09.00,2003.09.00,4555.0
1602,2005.03.27.R,Reported 27-Mar-2005,2005.0,Unprovoked,AUSTRALIA,New South Wales,NaN,Surfing,Chris Parker,M,...,N,NaN,NaN,"Sunday Age, 3/27/2005",2005.03.27.R-Parker.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2005.03.27.R,2005.03.27.R,4701.0
4738,1941.08.21.R,Reported 21-Aug-1941,1941.0,Provoked,USA,New York,Montauk,Fishing,Captain Jack Kelly,NaN,...,N,NaN,NaN,"New York Times, 8/21/1941",1941.08.21.R-Kelly.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1941.08.21.R,1941.08.21.R,1565.0
5443,1907.10.16.R,Reported 16-Oct-1907,1907.0,Unprovoked,CHINA,Hong Kong,"Sharp Peak, Sai Kung Peninsula, New Territories",Fishing,fishermen,M,...,N,NaN,Shark involvement probable,"Dawson Daily News, 11/20/1907",1907.10.16.R-HongKong.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1907.10.16.R,1907.10.16.R,860.0
1762,2003.07.00.a,Jul-2003,2003.0,Unprovoked,NEW ZEALAND,Cook Islands,Beveridge Reef,Snorkeling,Allen --,M,...,N,NaN,Grey reef shark,R. & J. Zorro; Noonsite.com,2003.07.00.a-BeveridgeReef.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2003.07.00.a,2003.07.00.a,4541.0


In [320]:
sharks.shape

(6303, 22)

In [321]:
sharks[sharks["case_number_1"]!=sharks["case_number_2"]].sample(21)[["case_number","case_number_1","case_number_2"]]

,case_number,case_number_1,case_number_2
6122,1808.06.01.R,1808.05.01.R,1808.06.01.R
3795,"1962,08.30.b",1962.08.30.b,"1962,08.30.b"
560,2014.05.04,2013.05.04,2014.05.04
5275,1900.00.00.R,1919.00.00.R,1900.00.00.R
4403,1952.08.05,1952.08.04,1952.08.05
34,2018.04.03,2018.04.02,2018.04.03
217,2016.09.15,2016.09.16,2016.09.15
5777,1884.04.38.R,1884.04.28.R,1884.04.38.R
314,2016.01.24.b,2015.01.24.b,2016.01.24.b
5386,1911.07.31.R,1911.07.31.T,1911.07.31.R


Viendo la compación de las columnas "case_number_1" y"case_number_2" con "case_number" se ve que la columna "case_number_1" contiene la misma información que la de "case_number_2" salvo en una pequeña cantidad de casos donde la columna "case_number" coincide con "case_number_2" por lo que se asume que la columna "case_number_1" no aporta nada de valor

In [322]:
sharks.drop(columns = ["case_number_1"],axis=1,inplace= True)

In [323]:
sharks[sharks["case_number"]!=sharks["case_number_2"]][["case_number","case_number_2","date"]].sample(5)

,case_number,case_number_2,date
5944,1864.05,1864.05.00,May-1864
25722,xx,NaN,NaN
5488,NaN,1905.09.06.R,Reported 06-Sep-1905
4949,1934.01.08.R,1934.02.08.R,Reported 08-Feb-1934
390,2015.07-10,2015.07.10,10-Jul-2015


Se comprueba que las columnas "case_number" y "case_number_2" son iguales salvo en unos casos donde comprobando con la columna date se ve que es la columna "case_number" la que es erronea

In [324]:
sharks.drop(columns = ["case_number"],axis=1,inplace= True)

In [325]:
sharks.sample(5)

,date,year,type,country,area,location,activity,name,sex,age,injury,fatal_(y/n),time,species,investigator_or_source,pdf,href_formula,href,case_number_2,original_order
6085,04-Jun-1832,1832.0,Unprovoked,AUSTRALIA,New South Wales,"South Head, Sydney",Fishing,Aboriginal female,F,NaN,Leg severed,N,NaN,NaN,"Sydney Herald, 6/11/1832",1832.06.04-AboriginalWoman.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1832.06.04,218.0
778,19-Jun-2012,2012.0,Invalid,USA,South Carolina,"Myrtle Beach, Horry County",Standing,Matthew Breen,M,16,"Laceration to foot. Injured by a stingray, not...",NaN,NaN,No shark involvement,C. Creswell,2012.06.19-MyrtleBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2012.06.19,5525.0
6191,Before 2012,0.0,Unprovoked,USA,Hawaii,Oahu,Diving,Ken O'Keefe,M,NaN,Minor laceration to hand,N,NaN,"Galapagos shark, 6'",K. O'keefe,ND-0136-Keefe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND-0136,112.0
2154,02-Nov-1998,1998.0,Provoked,JAPAN,Southern Japan,460 miles off Iwakuni,Fishing for tuna,Tadashi Kodama,M,52,PROVOKED INCIDENT Knee bitten by shark trap...,N,NaN,6' shark,"Deseret News, 11/3/1998, p.A7",1998.11.02.b-Kodama.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1998.11.02.b,4149.0
2731,05-Jun-1988,1988.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Swimming,Jason Jones,M,12,Four lacerations to his ankle,N,12h00,NaN,"Orlando Sentinel, 6/7/1988, p.D2",1988.06.05-JasonJones.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1988.06.05,3572.0


In [326]:
sharks.isna().sum()

date                         1
year                         3
type                         5
country                     51
area                       456
location                   541
activity                   545
name                       211
sex                        566
age                       2832
injury                      29
fatal_(y/n)                540
time                      3355
species                   2839
investigator_or_source      18
pdf                          1
href_formula                 2
href                         1
case_number_2                1
original_order               1
dtype: int64

Hasta aquí se han limpiado del DataFrame todas aquellas filas con todos los valores NaN y todas aquellas columnas que no aportan nada de información útil, a continuación se va a exportar el DataFrame modificado a un .csv

In [327]:
#sharks.to_csv("sharks.limpios",index=False)